In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle
from pyhere import here

SCOPES = ["https://mail.google.com/"]  # full access
# https://www.googleapis.com/auth/gmail.readonly
creds = None
if os.path.exists("token.pickle"):
    with open("token.pickle", "rb") as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            here("credentials.json"), SCOPES
        )
        creds = flow.run_local_server(port=0)
    with open("token.pickle", "wb") as token:
        pickle.dump(creds, token)

service = build("gmail", "v1", credentials=creds)
results = (
    service.users()
    .messages()
    .list(userId="me", maxResults=10, q="from:robinhood.com")
    .execute()
)
messages = results.get("messages", [])
for message in messages:
    print(f"Message ID: {message}")

In [ ]:
# get the message content
message = service.users().messages().get(userId="me", id=messages[0]["id"]).execute()
print(message)
print(list(message.keys())) 

In [ ]:
import base64
class Message:
    def __init__(self, message):
        self.message = message
        self.snippet = message["snippet"]
        self.get_headers()
        self.body = base64.urlsafe_b64decode(
            message["payload"]["parts"][0]["body"]["data"]
        ).decode("utf-8")

    def get_headers(self):
        self.headers = {
            header["name"]: header["value"]
            for header in self.message["payload"]["headers"]
        }
        self.subject = self.headers["Subject"]
        self.from_ = self.headers["From"]
        self.to = self.headers["To"]
        self.date = self.headers["Date"]
        return self.headers

    def __str__(self):
        return (
            f"From={self.from_},\n"
            f"To={self.to},\n"
            f"Date={self.date},\n"
            f"Subject={self.subject}\n"
            f"snippet={self.snippet}\n"
            f"Body:\n\n{self.body}"
        )
    def delete(self):
        service.users().messages().delete(userId="me", id=self.message["id"]).execute()

    # todo : get what is in the deleted box
m = Message(message)
print(m)
m.delete()
print("---")
m = Message(message)
print(m)
# for key in message.keys():
#     print(key, message[key])
# print("---")

# for key in message["payload"].keys():
#     print(key, message["payload"][key])

# for part in message["payload"]["parts"]:
#     print(part)
# print("---")

# for header in message["payload"]["headers"]:
#     print(header)

# print("---")
# import base64

# # decode the message content
# message_content = message["payload"]["parts"][0]["body"]["data"]
# message_content = base64.urlsafe_b64decode(message_content).decode("utf-8")
# print(message_content)